In [5]:
import torch
from pytorch_pretrained_bert import BertTokenizer

In [10]:
import pandas as pd
import numpy as np

In [6]:
### bert tokenizer
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.

Using cache found in /home/kjwspecial/.cache/torch/hub/huggingface_pytorch-transformers_master


In [7]:
# 1. 모든 sentence에 CLS, SEP 붙인다.
def get_token(text):
    text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    return tokenized_text

# 2. token index로 변환, padding 한다.
def get_ids(tokenized_text, max_length):
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = indexed_tokens + [0]*(max_length - len(indexed_tokens))
    return input_ids

# 3. segment_ids를 만든다.
def get_mask(input_ids):
    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks

def df_one_hot_encoded(df,column_name):
    df_one_hot_encoded = pd.get_dummies(df[column_name])
    target = np.array(df_one_hot_encoded)
    return target

In [8]:
def train_test_setting(df,name):
    '''
        max_len : description 최대길이
        description : token 변환
        
        df.OFFENSE_TYPE_ID[i] = get_token(ID)
        리스트를 copy해서 하기때문에 느린듯, 권장X
    '''
    #[CLS] [SEP] 붙임, token 변환
    description = df["description"]
    description = [get_token(x) for x in description]
    
    #max_len
    max_len=0
    for ID in description:
        if(max_len < len(ID)):
            max_len=len(ID)       
    
    #convert Id + padding
    input_ids = [get_ids(x,max_len) for x in description]
    
    #Attention_masks
    attention_masks = []
    attention_masks = get_mask(input_ids)
    
    #labels
    one_hot_target = df_one_hot_encoded(df,name)
    target = np.argmax(one_hot_target,axis=1)
    
    # class수
    num_classes=len(df[name].unique())
    
    return input_ids, target,  attention_masks, num_classes , description

In [3]:
# ## concat한것
def concat_text(sentence):
    vocab = list(tokenizer.vocab.keys())
    concat_text=''
    for word in sentence:
        if word ==0:
            break
        if vocab[word][0]!='#':
            concat_text += ' '+vocab[word]
        else:
            concat_text +=''+vocab[word][2:]
    return concat_text